# Villin subdomain HP-35

In [1]:
import molsysmt as msm
molsys = msm.convert('pdbid:1VII')
molsys = msm.build.remove_hydrogens(molsys)
molsys = msm.build.add_terminal_cappings(molsys, N_terminal='ACE', C_terminal='NME')
msm.convert(molsys, to_form='villin_hp35.pdb')

'villin_hp35.pdb'

In [22]:
from openmm.app import PDBFile, Modeller, ForceField, PME, HBonds, Simulation, StateDataReporter, DCDReporter
from openmm import unit, Vec3, LangevinIntegrator, Platform
import numpy as np
from sys import stdout

In [3]:
pdb = PDBFile('villin_hp35.pdb')

In [4]:
modeller = Modeller(pdb.topology, pdb.positions)

In [5]:
modeller.topology

<Topology; 1 chains, 38 residues, 299 atoms, 305 bonds>

In [6]:
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [7]:
_ = modeller.addHydrogens(forcefield, pH=7.4)

In [8]:
modeller.topology

<Topology; 1 chains, 38 residues, 605 atoms, 611 bonds>

In [9]:
clearance = 14.0*unit.angstroms
max_size = max(max((pos[i] for pos in modeller.positions))-min((pos[i] for pos in modeller.positions)) for i in range(3))
vectors = Vec3(1.0, 0, 0), Vec3(1.0/3.0, 2.0*np.sqrt(2.0)/3.0,0.0), Vec3(-1.0/3.0, np.sqrt(2.0)/3.0, np.sqrt(6.0)/3.0)
box_vectors = [(max_size+clearance)*v for v in vectors]

modeller.addSolvent(forcefield, model='tip3p', boxVectors = box_vectors, ionicStrength=0.0*unit.molar,
                    positiveIon='Na+', negativeIon='Cl-')

In [10]:
modeller.topology

<Topology; 3 chains, 1824 residues, 5959 atoms, 4179 bonds>

In [19]:
modeller.topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=4.247429394721985, y=0.0, z=0.0), Vec3(x=1.4158097982406617, y=4.004514836825319, z=0.0), Vec3(x=-1.4158097982406617, y=2.0022574184126594, z=3.4680115785224213)), unit=nanometer)

In [11]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.2*unit.nanometer, constraints=HBonds)

In [12]:
integrator = LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [13]:
platform = Platform.getPlatformByName('CUDA')

In [14]:
simulation = Simulation(modeller.topology, system, integrator, platform)

In [15]:
simulation.context.setPositions(modeller.positions)

In [16]:
simulation.minimizeEnergy()

In [17]:
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))

In [18]:
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-80509.98836319539,255.99131514158285
2000,-76550.87898819539,291.59431944193705
3000,-75694.58211319539,298.8741416370853
4000,-75722.64461319539,294.2623969033308
5000,-75519.58211319539,302.8573298379399
6000,-75070.39461319539,302.4441522882273
7000,-75641.73836319539,301.49771771886793
8000,-75845.83211319539,296.6954947047418
9000,-75806.22273819539,292.82596840661444
10000,-75876.59773819539,298.6811694473404


In [20]:
simulation.context.setTime=0.0*unit.picoseconds

In [23]:
simulation.reporters=[]
simulation.reporters.append(StateDataReporter(stdout, 50000, step=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('traj.dcd', 50000))

In [24]:
simulation.step(1000000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
50000,-75615.92586319539,300.62087745305274
100000,-76215.37898819539,300.0731393561469
150000,-76222.42586319539,296.7495471204649
200000,-76007.73836319539,303.37717046033634
250000,-76108.97273819539,297.7135757702844
300000,-75673.17586319539,304.73867920233965
350000,-75706.53523819539,294.68913204803897
400000,-75466.61336319539,294.862173863917
450000,-75869.09773819539,299.88229937494737
500000,-75661.64461319539,291.57427657142415
550000,-75276.89461319539,295.9157228065551
600000,-75741.08211319539,297.37899711121554
650000,-75703.26961319539,303.38964276696333
700000,-75761.70711319539,303.32573927267737
750000,-75538.53523819539,301.55669794673935
800000,-75882.78523819539,303.52608664924355
850000,-75630.92586319539,307.80926807453903
900000,-74934.30086319539,301.22307156683655
950000,-75899.86336319539,301.81332893154377
1000000,-75676.92586319539,302.56028063118106


In [ ]:
pdb = PDBFile('input_exercise2.pdb')
forcefield = ForceField('amber99sb.xml', 'amber99_obc.xml')
print "Building Model.."
modeller = Modeller(pdb.topology, pdb.positions)
print('Adding hydrogens...')
modeller.addHydrogens(forcefield)
print "Creating System"
system = forcefield.createSystem(modeller.topology, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 91/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
print "Using Platform:", simulation.context.getPlatform().getName()
simulation.context.setPositions(modeller.positions)